In [1]:
import requests
import json
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials


SPOTIFY_TOKEN_SCOPE = 'playlist-modify-private'
SPOTIFY_USERNAME = '42sw3i9k94fjxzgw00ug7kaza'
SPOTIFY_CLIENT_ID = 'f6768eaff74e41f8b6a0b42341acd841'
SPOTIFY_CLIENT_SECRET = '20fa102ef869490f87714d58803a6ed2'

DEEZER_ID = '1394580944'


track_ids = []


def get_deezer_user_data(user_id) -> dict:
    user_playlists = requests.get("https://api.deezer.com/user/" + user_id + "/playlists").json()
    return user_playlists   

def get_playlist_tracks_deezer(user_playlists, playlist_name) -> list:
    more_tracks = True
    loved_tracks_deezer = []
#     Get first page of tracklist
    for playlist in user_playlists["data"]:
        if playlist["title"].lower() == playlist_name.lower():
            tracklist_url = playlist["tracklist"]
            tracklist = requests.get(tracklist_url).json()
#     Get next pages of tracklist
    while more_tracks:
        for track in tracklist["data"]:
            loved_tracks_deezer.append([track["title"], track["artist"]["name"]])
        if "next" in tracklist.keys():
            tracklist = requests.get(tracklist["next"]).json()
        else:
            more_tracks = False
    return loved_tracks_deezer

def auth_spotify() -> spotipy.client.Spotify:
    scc = spotipy.oauth2.SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)    
    token = util.prompt_for_user_token(SPOTIFY_USERNAME, scope=SPOTIFY_TOKEN_SCOPE, 
                                       client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET, 
                                       redirect_uri="http://localhost/")
    sp_client = spotipy.client.Spotify(client_credentials_manager=scc, auth=token)
    return sp_client

def search_tracks_on_spotify(sp_client, tracklist) -> list:
    ids = []
    not_found = []
    for track in tracklist:
        q = track[0] + ' ' + track[1]
        results = sp_client.search(q, limit=10, offset=0, type='track', market=None)
        if len(results["tracks"]["items"]) > 0:
            ids.append(results["tracks"]["items"][0]["id"])
        else:
            not_found.append(q)
        
    return ids, not_found

def add_tracks_to_library(sp_client, ids) -> None:
    for i,track_id in enumerate(ids):
        print(str(i) + ": " + track_id)
        sp_client.current_user_saved_tracks_add([track_id])
        

def find_playlist_id(sp_client, name) -> str:
    results = sp_client.current_user_playlists(limit=50, offset=0)
    for playlist in results["items"]:
        if playlist["name"].lower() == name:
            return playlist["id"]

        
def add_to_playlist_spotify(sp_client, tracklist, playlist_id) -> None:
    for track in tracklist:
        print(track)
        sp_client.user_playlist_add_tracks(SPOTIFY_USERNAME, playlist_id, track)


def run_all():
    user_data = get_deezer_user_data(DEEZER_ID)
    tracklist = get_playlist_tracks_deezer(user_data, "all time favourites")
    sp_client = auth_spotify()
    ids, not_found = search_tracks_on_spotify(sp_client, tracklist)
    track_ids = ids
    playlist_id = find_playlist_id(sp_client, "all time favourites")
    add_to_playlist_spotify(sp_client, ids, playlist_id)
#     add_tracks_to_library(sp_client, ids)
    print(not_found)
    

In [51]:
print(track_ids)

[]


In [2]:
run_all()

5kBUIe8UcUUGLzP4hoc8XJ


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/users/42sw3i9k94fjxzgw00ug7kaza/playlists/6kVjKrJX4Bb1cUgDwGbtGZ/tracks:
 Invalid track uri: spotify:track:5

In [56]:
sp_client = auth_spotify()
find_playlist_id(sp_client, "meh")

[{'collaborative': False, 'external_urls': {'spotify': 'https://open.spotify.com/playlist/6kVjKrJX4Bb1cUgDwGbtGZ'}, 'href': 'https://api.spotify.com/v1/playlists/6kVjKrJX4Bb1cUgDwGbtGZ', 'id': '6kVjKrJX4Bb1cUgDwGbtGZ', 'images': [], 'name': 'All Time Favourites', 'owner': {'display_name': 'Tihan Pelser', 'external_urls': {'spotify': 'https://open.spotify.com/user/42sw3i9k94fjxzgw00ug7kaza'}, 'href': 'https://api.spotify.com/v1/users/42sw3i9k94fjxzgw00ug7kaza', 'id': '42sw3i9k94fjxzgw00ug7kaza', 'type': 'user', 'uri': 'spotify:user:42sw3i9k94fjxzgw00ug7kaza'}, 'primary_color': None, 'public': True, 'snapshot_id': 'MSxhMTE1Y2RmY2UwOTRlOWExNGJlNzdmOGNjYTRhZTA2NGY5OGM2ODZj', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/6kVjKrJX4Bb1cUgDwGbtGZ/tracks', 'total': 0}, 'type': 'playlist', 'uri': 'spotify:playlist:6kVjKrJX4Bb1cUgDwGbtGZ'}]
